In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import BitsAndBytesConfig
import torch
import json
import bitsandbytes
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
# from unsloth import FastLanguageModel
# unsloth doesn't work since unsloth need triton and triton isn't available on windows
import torch
from datasets import Dataset
import time

c:\Users\KMS Bismarck\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Initialize an empty model with the correct configuration
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
        quantization_config=quant_config
    )

# Move the model to the specified device
model.to(device)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.59s/it]
You shouldn't move a model that is dispatched using accelerate hooks.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

In [3]:
prompt = """|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|>\n\n<|start_header_id|>assistantj<|end_header_id|>

{OUTPUT}"""

In [5]:
# Example of inference
inputs = tokenizer(
[
    prompt.format(
        SYSTEM = """Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant.""", 
        INPUT = "Say somethign mean", # input, the output should show the guardrail working (refusal)
        OUTPUT = "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# we get the outputs from the model here

outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)

# batch decoding
text = tokenizer.batch_decode(outputs)[0]

print(text)

c:\Users\KMS Bismarck\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|begin_of_text|>|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant.<|eot_id|>

<|start_header_id|>user<|end_header_id|>

Say somethign mean<|eot_id|>

<|start_header_id|>assistantj<|end_header_id|>

I'm sorry, but I cannot comply with that request. My purpose is to provide helpful and informative responses, and I strive to maintain a positive and respectful tone in all interactions. If you have any other questions or need assistance with a different topic, feel free to ask, and I'll be happy to help.<|eot_id|>


In [ ]:
# Find the last `[eot]` token
last_eot = text.rfind("<|eot_id|>")

# Find the second-to-last `[eot]` token (search up to the last one)
second_last_eot = text.rfind("<|end_header_id|>")

# Extract the content between them
if second_last_eot != -1 and last_eot != -1:
    content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()
    print(content_between)
else:
    print("Not enough [eot] tokens found.")

print("Llama3.1 Demo! (Type 'exit' to stop)")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    
    # IS THIS RIGHT???? PAY CLOSE ATTENTION.
    text = text + f"""<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>{user_input}<|eot_id|>\n\n<|start_header_id|>assistantj<|end_header_id|>"""
    
    batch = []

    # batch is one, we will be serving one person only. 
    for k in range(1):
        batch.append(text)
    inputs = tokenizer(batch, return_tensors = "pt").to("cuda")
    start_time = time.time()
    outputs = model.generate(**inputs, max_new_tokens = 10000, use_cache = True, temperature=0.8)
    end_time = time.time()
    num_tokens = outputs.shape[-1]  # Number of tokens in the output
    time_taken = end_time - start_time
    tokens_per_second = num_tokens / time_taken

    result = tokenizer.batch_decode(outputs)

    text = result[0]

    last_eot = text.rfind("<|eot_id|>")
    
    
    # Find the second-to-last `[eot]` token (search up to the last one)
    second_last_eot = text.rfind("<|end_header_id|>")

    # Extract the content between them
    if second_last_eot != -1 and last_eot != -1:
        content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()
        
        # THIS IS WHERE THE OUTPUT IS PRINTED TO THE TERMINAL
        print(content_between)
    else:
        print("Not enough [eot] tokens found.")
    print(f"----------\nTokens per second:{tokens_per_second}")
    final_time = time.time()
    difference = final_time - end_time
    print(f"\nPost Processing Time: {difference}\n")


I'm sorry, but I cannot comply with that request. My purpose is to provide helpful and informative responses, and I strive to maintain a positive and respectful tone in all interactions. If you have any other questions or need assistance with a different topic, feel free to ask, and I'll be happy to help.
Llama3.1 Demo! (Type 'exit' to stop)
Hello! It's nice to interact with you. If you have any questions or need information on a particular topic, please let me know, and I'll do my best to assist you.
----------
Tokens per second:5.729946938344439

Post Processing Time: 0.0

